# Amazon Bestseller Analysis: Data Cleaning

### Notebook 02: Python Data Cleaning

This is the next notebook in the Amazon Bestseller Analysis project where I will clean and preprocess the bestseller data collected from Amazon. The main tasks are:
    
1. Load and combine data from today's scraper run
2. Clean and standardise titles (remove formatting, handle series information)
3. Handle missing format data
4. Standardise categories and age ranges
5. Create derived fields (price bands, age groups)
6. Export cleaned data for analysis

I have broken down the age ranges into distinct segments (6-8, 9-12, 12-14, 14-18) to allow for nuanced analysis of reading progression and format preferences as young readers mature, which I've found particularly valuable when considering series development and format strategies.

To start, loading the necessary libraries and files:

In [85]:
# Import necessary libraries
import pandas as pd
import numpy as np
import os
import glob
import re
from datetime import datetime


In [86]:
today = datetime.now().strftime("%Y%m%d")
raw_file_path = f'../data/raw/daily_bestsellers/bestsellers_{today}.csv'
master_file_path = '../data/processed/master_bestsellers.csv'

print(f"Processing data for {today}")
print(f"Loading raw data from: {raw_file_path}")

# Load today's data
df_today = pd.read_csv(raw_file_path)
print(f"Loaded {len(df_today)} records from today's scrape")

Processing data for 20250119
Loading raw data from: ../data/raw/daily_bestsellers/bestsellers_20250119.csv
Loaded 239 records from today's scrape


Now, I want to check for any immediate issues:

In [87]:
print("Dataset Info:")
print(df_today.info())

print("Missing Values:")
print(df_today.isnull().sum())

print("Value Counts for Format:")
print(df_today['format'].value_counts(dropna=False))

print("Value Counts for Category:")
print(df_today['category'].value_counts())

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239 entries, 0 to 238
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rank              239 non-null    int64  
 1   title             239 non-null    object 
 2   author            212 non-null    object 
 3   price             238 non-null    float64
 4   format            239 non-null    object 
 5   rating            232 non-null    float64
 6   review_count      232 non-null    float64
 7   isbn10            181 non-null    object 
 8   isbn13            200 non-null    float64
 9   page_count        217 non-null    float64
 10  category          239 non-null    object 
 11  target_age_range  167 non-null    object 
 12  timestamp         239 non-null    object 
 13  asin              239 non-null    object 
 14  product_url       239 non-null    object 
 15  image_url         239 non-null    object 
 16  category_ranks    119 non-null

### Changing Format of Data

In [88]:
# Convert integer columns from float
integer_columns = ['review_count', 'page_count', 'isbn13']
for col in integer_columns:
    df_today[col] = df_today[col].fillna(0)
    df_today[col] = df_today[col].astype(int)


In [89]:
### Standardise date range

In [90]:
def extract_age_range(age_str):
    """Extract age range from string formats like '1-2', '7+', etc."""
    if pd.isna(age_str):
        return None, None
    
    age_str = str(age_str).lower().strip()
    
    # Handle '+' format (e.g., '7+')
    if '+' in age_str:
        start_age = int(''.join(filter(str.isdigit, age_str)))
        return start_age, None
    
    # Handle range format (e.g., '1-2', '7 to 9')
    numbers = [int(n) for n in ''.join(c if c.isdigit() else ' ' for c in age_str).split()]
    if len(numbers) >= 2:
        return numbers[0], numbers[1]
    elif len(numbers) == 1:
        return numbers[0], None
        
    return None, None

def map_to_standard_range(start_age, end_age):
    """Map extracted ages to standard publishing ranges"""
    standard_ranges = [
        (0, 2, '0-2'),
        (3, 5, '3-5'),
        (6, 8, '6-8'),
        (9, 12, '9-12'),
        (13, 14, '13-14'),
        (15, 18, '15-18')
    ]
    
    if start_age is None:
        return None
        
    # Handle '+' format
    if end_age is None:
        if start_age <= 2:
            return '0-2'
        elif start_age <= 5:
            return '3-5'
        elif start_age <= 8:
            return '6-8'
        elif start_age <= 12:
            return '9-12'
        elif start_age <= 14:
            return '13-14'
        else:
            return '15-18'
            
    # Handle range format
    avg_age = (start_age + end_age) / 2
    for low, high, range_str in standard_ranges:
        if low <= avg_age <= high:
            return range_str
            
    # Default to closest range
    if avg_age < 0:
        return '0-2'
    elif avg_age > 18:
        return '15-18'
    
    return None

def standardise_age_range(row):
    """Create standardised age ranges based on target_age_range or category"""
    # First try to use target_age_range
    if pd.notna(row['target_age_range']):
        start_age, end_age = extract_age_range(row['target_age_range'])
        if start_age is not None:
            std_range = map_to_standard_range(start_age, end_age)
            if std_range:
                return std_range
    
    # If no valid age range found, use category
    category = str(row['category']).lower()
    category_mappings = {
        'baby': '0-2',
        'toddler': '0-2',
        'preschool': '3-5',
        'kindergarten': '3-5',
        'early reader': '6-8',
        'ages 6-8': '6-8',
        'middle grade': '9-12',
        'ages 9-12': '9-12',
        'young teen': '13-14',
        'ages 12-14': '13-14',
        'teen': '15-18',
        'young adult': '15-18',
        'ages 14-18': '15-18'
    }
    
    for key, value in category_mappings.items():
        if key in category:
            return value
            
    return None

# Apply age range standardization
df_today['age_range_std'] = df_today.apply(standardise_age_range, axis=1)

# Create age group categories
age_group_mapping = {
    '0-2': '0-2',
    '3-5': '3-5',
    '6-8': '6-8',
    '9-12': '9-12',
    '13-14': '13-14',
    '15-18': '15-18'
}
df_today['age_range_std'] = df_today['age_range_std'].map(age_group_mapping)
#df_today = df_today.drop(['target_age_range','target_age_group'], axis =1)
# Print age range distribution
print("\nAge Range Distribution:")
print(df_today['age_range_std'].value_counts(dropna=False))


Age Range Distribution:
age_range_std
NaN      68
9-12     46
3-5      41
6-8      37
15-18    30
0-2      17
Name: count, dtype: int64


### Sorting the category columns

In [91]:
# Rename basic ranking columns for clarity
df_today = df_today.rename(columns={
    'rank': 'main_category_rank',
    'category': 'main_category'
})

In [92]:
def parse_category_ranking(category_str):
    """Parse category ranking string into separate categories and ranks"""
    if pd.isna(category_str):
        return pd.Series({
            'category_1': None, 
            'category_1_rank': pd.NA,
            'category_2': None, 
            'category_2_rank': pd.NA,
            'category_3': None, 
            'category_3_rank': pd.NA
        })
        
    # Split into individual category-rank pairs
    categories = [x.strip() for x in str(category_str).split('|')]
    
    # Initialise results dictionary
    results = {}
    
    # Process up to 3 categories
    for i in range(3):
        # Set default values
        results[f'category_{i+1}'] = None
        results[f'category_{i+1}_rank'] = pd.NA
        
        # If data for this position, process it
        if i < len(categories):
            cat_data = categories[i].strip()
            
            # Extract rank (first number in string)
            rank_match = re.match(r'(\d+)', cat_data)
            if rank_match:
                results[f'category_{i+1}_rank'] = pd.NA if rank_match is None else int(rank_match.group(1))
                
                # Extract category (everything after 'in ')
                category = re.search(r' in (.+)', cat_data)
                if category:
                    results[f'category_{i+1}'] = category.group(1).strip()
    
    # Convert to series with proper types
    series = pd.Series(results)
    for i in range(1, 4):
        if pd.notna(series[f'category_{i}_rank']):
            series[f'category_{i}_rank'] = int(series[f'category_{i}_rank'])
    return series

In [93]:
# Apply the parsing function and add new columns
df_today = df_today.rename(columns={'rank': 'main_category_rank', 'category': 'main_category'})
parsed_categories = df_today['category_ranks'].apply(parse_category_ranking)
df_today = pd.concat([df_today, parsed_categories], axis=1)

In [94]:
df_today.head()

,main_category_rank,title,author,price,format,rating,review_count,isbn10,isbn13,page_count,...,product_url,image_url,category_ranks,age_range_std,category_1,category_1_rank,category_2,category_2_rank,category_3,category_3_rank
0,1,Guess How Much I Love You: The beloved classic...,Sam McBratney,4.0,Board book,4.8,8752,NaN,9781406358780,32,...,https://www.amazon.co.uk/Guess-How-Much-Love-Y...,https://images-eu.ssl-images-amazon.com/images...,1 in Fiction About Emotions & Feelings for Chi...,NaN,Fiction About Emotions & Feelings for Children,1,Classics for Children,1,Children's Books on Valentine's Day,1
1,2,Dear Zoo: The Lift-the-flap Preschool Classic,Rod Campbell,4.0,Board book,4.8,30581,1529074932,9781529074932,18,...,https://www.amazon.co.uk/Dear-Zoo-Anniversary-...,https://images-eu.ssl-images-amazon.com/images...,2 in Classics for Children | 2 in Children's E...,0-2,Classics for Children,2,Children's Early Learning Books on Size & Shape,2,Children's Fiction Books on Animals,6
2,3,The Tiger Who Came to Tea,Judith Kerr,4.0,Paperback,4.8,15641,0007368380,9780007215997,32,...,https://www.amazon.co.uk/Tiger-Who-Came-Tea/dp...,https://images-eu.ssl-images-amazon.com/images...,2 in Children’s Daily Activity Fiction Books |...,3-5,Children’s Daily Activity Fiction Books,2,Classics for Children,3,Doctors & Medicine Humour,4
3,4,We're Going on a Bear Hunt: The bestselling cl...,Michael Rosen,4.0,Paperback,4.8,4779,0744523230,9780744523232,40,...,https://www.amazon.co.uk/Were-Going-Bear-Micha...,https://images-eu.ssl-images-amazon.com/images...,2 in Children's Early Learning Books on Words ...,3-5,Children's Early Learning Books on Words,2,Classics for Children,4,Activity Books for Children,12
4,5,Oh Dear!: A Lift-the-flap Farm Book from the C...,Rod Campbell,4.0,Board book,4.8,2619,1529097886,9781529097887,18,...,https://www.amazon.co.uk/Oh-Dear-Rod-Campbell/...,https://images-eu.ssl-images-amazon.com/images...,1 in Children's Books on Easter | 2 in Childre...,NaN,Children's Books on Easter,1,Children's Books on Country & Farm Life,2,Children's Books on Farm Animals,2


### Clean and standardise text fields

In [95]:
# Clean titles
df_today['clean_title'] = df_today['title'].str.strip()
df_today['clean_title'] = df_today['clean_title'].str.replace('"', '"').str.replace('"', '"')

# Clean author names
df_today['author'] = df_today['author'].str.strip()
df_today['author'] = df_today['author'].str.replace(r'\s+', ' ', regex=True)

### 5. Format standardization

In [96]:
format_mapping = {
    'paperback': 'Paperback',
    'hardcover': 'Hardcover',
    'hard cover': 'Hardcover',
    'board book': 'Board Book',
    'board': 'Board Book',
    'kindle': 'Digital',
    'ebook': 'Digital',
    'e-book': 'Digital',
    'audiobook': 'Audio',
    'audio book': 'Audio'
}

df_today['standardised_format'] = df_today['format'].str.lower().map(format_mapping)

# 6. Add processing metadata

In [97]:
df_today['processing_date'] = today
df_today['data_batch'] = f'batch_{today}'

In [98]:
### 7. Validation checks before appending

In [99]:

validation_issues = []

# Check for critical issues
if len(df_today) == 0:
    validation_issues.append("No records found in today's data")
    
if df_today['price'].isnull().all():
    validation_issues.append("All prices are missing")
    
if df_today['standardised_format'].isnull().all():
    validation_issues.append("All formats are missing")

# Check for suspicious patterns
    
if df_today['price'].max() > 100:
    validation_issues.append(f"Suspicious high price found: {df_today['price'].max()}")

if len(validation_issues) > 0:
    print("\nWARNING: Data validation issues found:")
    for issue in validation_issues:
        print(f"- {issue}")
    print("\nPlease review the issues before proceeding.")
else:
    print("Data validation passed")


Data validation passed


In [100]:
### 8. Load and append to master dataset

In [101]:
if os.path.exists(master_file_path):
    # Load existing master data
    df_master = pd.read_csv(master_file_path)
    print(f"Loaded {len(df_master)} existing records from master dataset")
    
    # Append new data
    df_master = pd.concat([df_master, df_today], ignore_index=True)
    
    # Remove duplicates based on business key (adjust columns as needed)
    df_master = df_master.drop_duplicates(
        subset=['clean_title', 'author', 'standardised_format', 'processing_date'],
        keep='last'
    )
else:
    print("No existing master dataset found. Creating new one.")
    df_master = df_today

# Save updated master dataset
df_master.to_csv(master_file_path, index=False)

Loaded 239 existing records from master dataset


### 9. Generate daily summary

In [102]:
print("\nDaily Processing Summary:")
print("-" * 50)
print(f"Date: {today}")
print(f"Records processed: {len(df_today)}")
print(f"Records in master dataset: {len(df_master)}")
print("\nFormat Distribution (Today):")
print(df_today['standardised_format'].value_counts())
print("\nPrice Statistics (Today):")
print(df_today['price'].describe())

# 10. Save daily summary to log
log_file_path = '../data/processed/processing_log.csv'
daily_summary = {
    'date': today,
    'records_processed': len(df_today),
    'total_records': len(df_master),
    'validation_issues': '; '.join(validation_issues) if validation_issues else 'None'
}

if os.path.exists(log_file_path):
    df_log = pd.read_csv(log_file_path)
else:
    df_log = pd.DataFrame(columns=['date', 'records_processed', 'total_records', 'validation_issues'])

df_log = pd.concat([df_log, pd.DataFrame([daily_summary])], ignore_index=True)
df_log.to_csv(log_file_path, index=False)

print("\nProcessing complete. Check processing_log.csv for historical records.")



Daily Processing Summary:
--------------------------------------------------
Date: 20250119
Records processed: 239
Records in master dataset: 433

Format Distribution (Today):
standardised_format
Paperback     119
Hardcover      42
Board Book     32
Name: count, dtype: int64

Price Statistics (Today):
count    238.000000
mean       7.152353
std        5.456755
min        0.990000
25%        4.000000
50%        5.985000
75%        7.925000
max       39.990000
Name: price, dtype: float64

Processing complete. Check processing_log.csv for historical records.


# Summary and Next Steps

## Data Cleaning Accomplished
In this notebook, I worked on comprehensive data preparation and cleaning. I began by standardising and cleaning book titles and extracting series information before handling missing format data through inference. To ensure consistency, I standardised categories and age ranges. The process included implementing data quality checks and saving reusable cleaning functions. 

Now, the resulting dataset has complete data for essential fields like title, author, price and format, with standardised formats and categories and normalised rankings within categories. I also created consistent age groupings across the data. Key improvements included filling missing format data using title and price inference, standardising age ranges across categories, adding normalised ranking metrics and extracting standardised series information. 

### Next Steps:
3. Price analysis ([03_price_analysis.ipynb](03_price_analysis.ipynb)): Create visualizations of price distributions across formats and categories
4. Genre analysis ([04_genre_analysis.ipynb](04_genre_analysis.ipynb)): Analyze category performance and age-range segmentation
5. Seasonal analysis ([05_seasonal_analysis.ipynb](05_seasonal_analysis)): Begin tracking temporal patterns (will become more valuable as I collect more data over time)